# Data cleaning
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for automating a data cleaning task. In a series of steps we show how to use EDSL to automatically suggest appropriate sense checks for a given dataset, convert those sense checks into survey questions, and then prompt a model to answer them to generate a new dataset of the data failing the checks.

EDSL is an open-source library for simulating surveys and experiments with AI agents and large language models. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

### Example data
We'll use some observations of ages as our dataset for cleaning. It's a list of random ages between 22 and 85 with some bad values mixed in:

In [1]:
ages = [
    84,
    62,
    79,
    57,
    59,
    55,
    68,
    66,
    47,
    54,
    76,
    33,
    74,
    56,
    47,
    24,
    23,
    38,
    38,
    54,
    51,
    84,
    71,
    46,
    38,
    26,
    50,
    56,
    62,
    39,
    31,
    52,
    69,
    84,
    69,
    48,
    48,
    23,
    65,
    54,
    78,
    51,
    69,
    77,
    75,
    76,
    26,
    44,
    61,
    32,
    70,
    24,
    74,
    22,
    32,
    24,
    80,
    65,
    36,
    42,
    84,
    66,
    40,
    85,
    28,
    22,
    67,
    25,
    70,
    77,
    53,
    69,
    64,
    27,
    61,
    68,
    68,
    78,
    0.99,
    83,
    58,
    33,
    46,
    43,
    50,
    85,
    28,
    82,
    50,
    61,
    66,
    32,
    45,
    70,
    56,
    50,
    43,
    30,
    43,
    55,
    33,
    72,
    43,
    43,
    -5,
    32,
    43,
    45,
    67,
    84,
    37,
    63,
    52,
    53,
    58,
    79,
    79,
    80,
    62,
    75,
    57,
    60,
    39,
    79,
    49,
    60,
    60,
    37,
    45,
    36,
    1050,
    73,
    70,
    56,
    39,
    58,
    69,
    77,
    68,
    84,
    78,
    48,
    31,
    74,
    27,
    55,
    56,
    66,
    35,
    39,
    57,
    47,
    29,
    24,
    47,
    60,
    43,
    37,
    84,
    64,
    28,
    22,
    37,
    71,
    77,
    76,
    84,
    63,
    76,
    58,
    41,
    72,
    22,
    63,
    78,
    49,
    82,
    69,
    "old",
    37,
    27,
    29,
    54,
    83,
    80,
    74,
    48,
    76,
    49,
    26,
    38,
    35,
    36,
    25,
    23,
    71,
    33,
    39,
    40,
    35,
    85,
    24,
    57,
    85,
    63,
    53,
    62,
    47,
    69,
    76,
    71,
    48,
    62,
    23,
    25,
    84,
    32,
    63,
    75,
    31,
    25,
    50,
    85,
    36,
    58,
    85,
    34,
    62,
    43,
    2,
    50,
    83,
    44,
    73,
    81,
    44,
    43,
    82,
    84,
    30,
    24,
    63,
    63,
    59,
    46,
    30,
    62,
    25,
    52,
    23,
]

### Constructing questions about the data
We start by creating a question prompting a model to draft some sense check questions for our data. EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the desired form of the response (multiple choice, free text, etc.). Here we select `QuestionList` in order to prompt the model to return a list of sense check question text. Note that we specially instruct the model to include a placeholder for a piece of data in each question text so that we can run them for each piece of data all at once:

In [2]:
import random

data_description = (
    "a list of ages (in years) of adult participants in a social science experiment"
)
sample_data = random.sample(ages, 5)
number_of_questions = 4

In [3]:
from edsl.questions import QuestionList

q1 = QuestionList(
    question_name="data_cleaning_questions",
    question_text=f"""Consider a dataset consisting of {data_description}.
    Here is a sample of the data: {sample_data}.
    Draft a set of appropriate sense checks for the data in this dataset, 
    formatted as a list of questions to be answered about each piece of data 
    individually and without reference to other data, using '<data>' as a 
    placeholder for the piece of data in each question text.""",
    max_list_items=number_of_questions,
)

We generate a response by calling the `run()` method for the question. This generates a dataset of `Results` that we can begin analyzing:

EDSL comes with [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html) as datasets, dataframes, JSON and other forms. We can inspect a list of all the components that are directly accessible:

In [5]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.data_cleaning_questions',
 'comment.data_cleaning_questions_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.data_cleaning_questions_system_prompt',
 'prompt.data_cleaning_questions_user_prompt',
 'question_options.data_cleaning_questions_question_options',
 'question_text.data_cleaning_questions_question_text',
 'question_type.data_cleaning_questions_question_type',
 'raw_model_response.data_cleaning_questions_raw_model_response']

Here we select just the answer (the draft data cleaning question texts) and print it in a table:

In [4]:
results = q1.run()

In [6]:
results.select("data_cleaning_questions").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .data_cleaning_questions                                                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ['Is <data> a non-negative integer?', 'Is <data> realistically within the human lifespan?', 'Is <data> greater  │
│ than or equal to 18?', 'Is <data> a whole number without decimal places?']                                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

### Parameterizing questions
Next we use the data cleaning question texts as inputs to a new question prompting the model to draft a list of appropriate answer options. This is done by creating a `Scenario` for each data cleaning question that we add to our new question when we run it ([learn more about using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html)):

In [7]:
data_cleaning_questions = results.select("data_cleaning_questions").to_list()[0]
data_cleaning_questions

['Is <data> a non-negative integer?',
 'Is <data> realistically within the human lifespan?',
 'Is <data> greater than or equal to 18?',
 'Is <data> a whole number without decimal places?']

In [8]:
from edsl.questions import QuestionList

q2 = QuestionList(
    question_name="data_cleaning_options",
    question_text="Consider a dataset consisting of "
    + data_description
    + "\nHere is a sample of the data: "
    + str(sample_data)
    + """Draft a list of appropriate answer options for the following sense check
    question about this data: {{ data_cleaning_question }}.
    Include 'Yes', 'No', and any other options that could add helpful context
    to flag a piece of data that is potentially problematic.""",
)

In [9]:
from edsl import Scenario

scenarios = [Scenario({"data_cleaning_question": q}) for q in data_cleaning_questions]
scenarios

[Scenario({'data_cleaning_question': 'Is <data> a non-negative integer?'}),
 Scenario({'data_cleaning_question': 'Is <data> realistically within the human lifespan?'}),
 Scenario({'data_cleaning_question': 'Is <data> greater than or equal to 18?'}),
 Scenario({'data_cleaning_question': 'Is <data> a whole number without decimal places?'})]

In [10]:
results = q2.by(scenarios).run()

In [11]:
results.select("data_cleaning_question", "data_cleaning_options").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario                                           ┃ answer                                                     ┃
┃ .data_cleaning_question                            ┃ .data_cleaning_options                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Is <data> a non-negative integer?                  │ ['Yes', 'No', 'Data is not an integer', 'Data is           │
│                                                    │ negative', 'Data is not a valid number']                   │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────┤
│ Is <data> a whole number without decimal places?   │ ['Yes', 'No', 'The data includes a non-integer value',     │
│                                                    │ 'The data contains an age below the minimum adult age',    │
│                                                    │ 'The data includes an unlikely age for a living person']   │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────┤
│ Is <data> greater than or equal to 18?             │ ['Yes', 'No', 'Data is invalid', 'Data is missing']        │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────┤
│ Is <data> realistically within the human lifespan? │ ['Yes', 'No', 'Unlikely', 'Typographical error',           │
│                                                    │ 'Outlier', 'Verify']                                       │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────┘

## Constructing a data cleaning survey
Next we can make any desired edits to the data cleaning questions, and then use them to create a `Survey` about the data that we prompt the model to answer ([learn more about constructing surveys](https://docs.expectedparrot.com/en/latest/surveys.html)). We use `QuestionMultipleChoice` to facilitate the use of the answer options that have been created:

In [12]:
import ast

df = results.to_pandas(remove_prefix=True)
df = df[["data_cleaning_question", "data_cleaning_options"]]

In [13]:
from edsl import QuestionMultipleChoice

formatted_questions = []

for index, row in df.iterrows():
    q = QuestionMultipleChoice(
        question_name="q" + str(index),
        question_text=row["data_cleaning_question"].replace("<data>", "{{ age }}"),
        question_options=ast.literal_eval(row["data_cleaning_options"]),
    )
    formatted_questions.append(q)

formatted_questions

[Question('multiple_choice', question_name = 'q0', question_text = 'Is {{ age }} a non-negative integer?', question_options = ['Yes', 'No', 'Data is not an integer', 'Data is negative', 'Data is not a valid number']),
 Question('multiple_choice', question_name = 'q1', question_text = 'Is {{ age }} a whole number without decimal places?', question_options = ['Yes', 'No', 'The data includes a non-integer value', 'The data contains an age below the minimum adult age', 'The data includes an unlikely age for a living person']),
 Question('multiple_choice', question_name = 'q2', question_text = 'Is {{ age }} greater than or equal to 18?', question_options = ['Yes', 'No', 'Data is invalid', 'Data is missing']),
 Question('multiple_choice', question_name = 'q3', question_text = 'Is {{ age }} realistically within the human lifespan?', question_options = ['Yes', 'No', 'Unlikely', 'Typographical error', 'Outlier', 'Verify'])]

In [14]:
from edsl import Survey

survey = Survey(formatted_questions)

We create `Scenario` objects for the individual ages that we will insert in the question texts when we run the survey:

In [15]:
scenarios = [Scenario({"age": age}) for age in ages]

In [16]:
results = survey.by(scenarios).run()

Here we inspect all the components of these new results, print a table of a set of them (optionally set `max_rows`), and then print a table where the results are filtered to cases for review, based on the responses to the questions (see the `filter` logic):

In [17]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.q0',
 'answer.q1',
 'answer.q2',
 'answer.q3',
 'comment.q0_comment',
 'comment.q1_comment',
 'comment.q2_comment',
 'comment.q3_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.q0_system_prompt',
 'prompt.q0_user_prompt',
 'prompt.q1_system_prompt',
 'prompt.q1_user_prompt',
 'prompt.q2_system_prompt',
 'prompt.q2_user_prompt',
 'prompt.q3_system_prompt',
 'prompt.q3_user_prompt',
 'question_options.q0_question_options',
 'question_options.q1_question_options',
 'question_options.q2_question_options',
 'question_options.q3_question_options',
 'question_text.q0_question_text',
 'question_text.q1_question_text',
 'question_text.q2_question_text',
 'question_text.q3_question_text',
 'question_type.q0_question_type',
 'question_type.q1_question_type',
 'question_type.q

In [18]:
(
    results.select("age", "q0", "q1", "q2", "q3").print(
        pretty_labels={
            "scenario.age": "Age",
            "answer.q0": data_cleaning_questions[0],
            "answer.q1": data_cleaning_questions[1],
            "answer.q2": data_cleaning_questions[2],
            "answer.q3": data_cleaning_questions[3],
        },
        format="rich",
        max_rows=10,
    )
)

Showing only the first 10 rows of 250 rows.


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃     ┃                          ┃ Is <data> realistically  ┃                          ┃                          ┃
┃     ┃ Is <data> a non-negative ┃ within the human         ┃ Is <data> greater than   ┃ Is <data> a whole number ┃
┃ Age ┃ integer?                 ┃ lifespan?                ┃ or equal to 18?          ┃ without decimal places?  ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 30  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 52  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 30  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 83  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 22  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 63  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 62  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 69  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 84  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
├─────┼──────────────────────────┼──────────────────────────┼──────────────────────────┼──────────────────────────┤
│ 72  │ Yes                      │ Yes                      │ Yes                      │ Yes                      │
└─────┴──────────────────────────┴──────────────────────────┴──────────────────────────┴──────────────────────────┘

In [19]:
(
    results.filter("q0 != 'Yes' or q1 != 'Yes' or q2 != 'Yes' or q3 != 'Yes'")
    .select("age")
    .print(pretty_labels={"scenario.age": "Bad data"}, format="rich")
)

┏━━━━━━━━━━┓
┃ Bad data ┃
┡━━━━━━━━━━┩
│ -5       │
├──────────┤
│ old      │
├──────────┤
│ 0.99     │
├──────────┤
│ 1050     │
├──────────┤
│ 2        │
└──────────┘

## Further exploration
This notebook can be readily edited and expanded for other data cleaning and data labeling purposes, or to add personas for AI agents answering the questions with relevant background and expertise. [Learn more about using AI agents for your EDSL surveys](https://docs.expectedparrot.com/en/latest/agents.html). 

Please see our [documentation page](https://docs.expectedparrot.com/) for examples of other methods and use cases and let us know if you have any questions!